In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/mnt/code/code/noc/class/zarr_visualization/xarray_leaflet/xarray_leaflet/xarray_leaflet.py:23: AccessorRegistrationWarning: registration of accessor <class 'xarray_leaflet.xarray_leaflet.LeafletMap'> under name 'leaflet' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  class LeafletMap(HasTraits):


In [12]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import matplotlib.pyplot as plt

In [13]:

url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = os.path.join(name, name, 'w001001.adf')

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [14]:
adffile = 'as_dem_30s/as_dem_30s/w001001.adf'
da = rioxarray.open_rasterio(adffile, masked=True)
da

<xarray.DataArray (band: 1, y: 8400, x: 15000)> Size: 504MB
[126000000 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 120kB 55.0 55.01 55.02 55.03 ... 180.0 180.0 180.0
  * y            (y) float64 67kB 60.0 59.99 59.98 ... -9.979 -9.988 -9.996
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [15]:
da = da.sel(band=1)
da.name = 'DEM'

In [16]:
da = da.chunk({'x': 1000, 'y': 1000})

In [17]:
type(da)

xarray.core.dataarray.DataArray

In [18]:
# da.plot()

In [19]:
m = Map(center=[40, 115], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

Map(center=[40, 115], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [20]:
l = da.leaflet.plot(m, colormap=plt.cm.terrain)
# l.interact(opacity=(0., 1.))

SSSSSSSSSSSS


Url()

In [20]:
layers_control = LayersControl(position='topright')
m.add_control(layers_control)

opacity_slider = FloatSlider(description='Opacity:', min=0, max=1, value=1)

def set_opacity(change):
    l.opacity = change['new']

opacity_slider.observe(set_opacity, names='value')
slider_control = WidgetControl(widget=opacity_slider, position='bottomleft')
m.add_control(slider_control)

In [15]:
da.leaflet.select()

In [12]:
da_selected = da.leaflet.get_selection()


In [13]:
if da_selected is not None:
    da_selected.plot.imshow()

In [14]:
da.leaflet.unselect()  # remove the selection buttons